In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
%matplotlib inline
from geopy.geocoders import Nominatim
from scipy.spatial.distance import pdist

print('Libraries imported.')

Libraries imported.


In [2]:
CLIENT_ID = 'JQYNPSOQXI12NK311020I2K4FAKMKEDV25CMLKTGSEUEVMUR' # your Foursquare ID
CLIENT_SECRET = 'XRECAKXLEOK324BN2Z3KDYNK1NBJ3UDYYS45PQAJ2LIM0BFX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JQYNPSOQXI12NK311020I2K4FAKMKEDV25CMLKTGSEUEVMUR
CLIENT_SECRET:XRECAKXLEOK324BN2Z3KDYNK1NBJ3UDYYS45PQAJ2LIM0BFX


In [3]:
LIMIT = 100 
cities = ['Cracow', 'Poznan', 'Warsaw', 'Wroclaw']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1e7931735,4bf58dd8d48988d1e9931735") # PIZZA PLACE CATEGORY ID
    results[city] = requests.get(url).json()


In [4]:
results

{'Cracow': {'meta': {'code': 200, 'requestId': '5e139e9e1e152c001b9cecf7'},
  'response': {'geocode': {'cc': 'PL',
    'center': {'lat': 50.06143, 'lng': 19.93658},
    'displayString': 'Krakow, Poland',
    'geometry': {'bounds': {'ne': {'lat': 50.125746929273035,
       'lng': 20.217017998898445},
      'sw': {'lat': 49.96750603553661, 'lng': 19.793890515115354}}},
    'longId': '72057594041022738',
    'slug': 'krakow-poland',
    'what': '',
    'where': 'cracow'},
   'groups': [{'items': [{'reasons': {'count': 0,
        'items': [{'reasonName': 'globalInteractionReason',
          'summary': 'This spot is popular',
          'type': 'general'}]},
       'referralId': 'e-0-4bbf90a2b492d13a4fdfa260-0',
       'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/musicvenue_jazzclub_',
           'suffix': '.png'},
          'id': '4bf58dd8d48988d1e7931735',
          'name': 'Jazz Club',
          'pluralName': 'Jazz Clubs',
       

In [5]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address','venue.id', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address','VenueID', 'Lat', 'Lng']

In [6]:
df_venues

{'Cracow':                             Name                          Address  \
 0              Harris Piano Jazz                  Rynek Główny 28   
 1                 Jazz Rock Cafe                    Sławkowska 12   
 2          Hard Rock Cafe Kraków  Main Market / Mariacki Square 9   
 3                The Piano Rouge                  Rynek Główny 46   
 4            Jazz Club U Muniaka                     Floriańska 3   
 5                      Drukarnia                   Nadwiślańska 1   
 6    Piec Art Acoustic Jazz Club                              NaN   
 7        Chicago Jazz Live Music                              NaN   
 8                   Klub Kwadrat                  Skarżyńskiego 1   
 9           Klub Muzyczny Awaria                     Mikołajska 9   
 10                  Carpe Diem 2                 ul. Sławkowska 6   
 11  Klub Studencki AGH "Filutek"                 Rostafińskiego 4   
 12              Kitsch Kazimierz                        Dajwor 16   
 
        

In [7]:
geolocator = Nominatim(user_agent="capstone")
maps= {}
for city in cities:
    location = geolocator.geocode(city + ', Poland')
    city_lat = location.latitude
    city_lng = location.longitude
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=12)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of music clubs in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of music clubs in Cracow =  13
Showing Top 100
Total number of music clubs in Poznan =  5
Showing Top 100
Total number of music clubs in Warsaw =  16
Showing Top 100
Total number of music clubs in Wroclaw =  6
Showing Top 100


Cracow

In [8]:
maps[cities[0]]

Poznan

In [9]:
maps[cities[1]]


Warsaw

In [10]:
maps[cities[2]]

Wroclaw

In [11]:
maps[cities[3]]

In [12]:
df_venues

{'Cracow':                             Name                          Address  \
 0              Harris Piano Jazz                  Rynek Główny 28   
 1                 Jazz Rock Cafe                    Sławkowska 12   
 2          Hard Rock Cafe Kraków  Main Market / Mariacki Square 9   
 3                The Piano Rouge                  Rynek Główny 46   
 4            Jazz Club U Muniaka                     Floriańska 3   
 5                      Drukarnia                   Nadwiślańska 1   
 6    Piec Art Acoustic Jazz Club                              NaN   
 7        Chicago Jazz Live Music                              NaN   
 8                   Klub Kwadrat                  Skarżyńskiego 1   
 9           Klub Muzyczny Awaria                     Mikołajska 9   
 10                  Carpe Diem 2                 ul. Sławkowska 6   
 11  Klub Studencki AGH "Filutek"                 Rostafińskiego 4   
 12              Kitsch Kazimierz                        Dajwor 16   
 
        

In [13]:
df_cracow = df_venues.get('Cracow')

In [14]:
df_wroclaw = df_venues.get('Wroclaw')

In [15]:
df_warsaw = df_venues.get('Warsaw')

In [16]:
df_poznan = df_venues.get('Poznan')

<h1>Cluster analysis </h1>

<h3>Cracow</h3>

In [17]:
from sklearn.cluster import KMeans

In [18]:
df_cluster = df_cracow.drop(['Name','Address', 'VenueID'], 1)

In [19]:
kclusters = 4

In [20]:
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_cluster)

In [21]:
kmeans.labels_[0:13]

array([2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 3, 0])

In [22]:
df_cracow.insert(0, 'Cluster Labels', kmeans.labels_)
df_cracow.head(2)

,Cluster Labels,Name,Address,VenueID,Lat,Lng
0,2,Harris Piano Jazz,Rynek Główny 28,4bbf90a2b492d13a4fdfa260,50.061703,19.935660
1,2,Jazz Rock Cafe,Sławkowska 12,4befe97724f19c747926f983,50.063866,19.937492


In [23]:
# create map
latitude=50.049683
longitude=19.944544
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(df_cracow['Lat'], df_cracow['Lng'],  df_cracow['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [24]:
df_cracow['Cluster Labels'].value_counts()

2    9
0    2
3    1
1    1
Name: Cluster Labels, dtype: int64

We will consider cluster 2

In [25]:
df_cracow_fin = df_cracow.loc[df_cracow['Cluster Labels'] == 2]

In [26]:
df_cracow_fin

,Cluster Labels,Name,Address,VenueID,Lat,Lng
0,2,Harris Piano Jazz,Rynek Główny 28,4bbf90a2b492d13a4fdfa260,50.061703,19.935660
1,2,Jazz Rock Cafe,Sławkowska 12,4befe97724f19c747926f983,50.063866,19.937492
2,2,Hard Rock Cafe Kraków,Main Market / Mariacki Square 9,4c28c33b9eb195213e982959,50.061442,19.938733
3,2,The Piano Rouge,Rynek Główny 46,4c7e5d5e3b22a1cdc388619e,50.062130,19.938760
4,2,Jazz Club U Muniaka,Floriańska 3,4f0231f9be7bbcd4f5cd8fd0,50.062189,19.939250
6,2,Piec Art Acoustic Jazz Club,NaN,4dfa5cced164bbe54616bab9,50.062661,19.934744
7,2,Chicago Jazz Live Music,NaN,52f6a5a5498e48263b2adaca,50.063970,19.937753
9,2,Klub Muzyczny Awaria,Mikołajska 9,4bfcd3a7e930952171f061ab,50.061266,19.942233
10,2,Carpe Diem 2,ul. Sławkowska 6,4be96c6ca117b713e28281c8,50.063289,19.937282


In [27]:
# create map
latitude=df_cracow_fin['Lat'].mean()
longitude=df_cracow_fin['Lng'].mean()
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=16)

# set color scheme for the clusters
#x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon in zip(df_cracow_fin['Lat'], df_cracow_fin['Lng']):
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        #popup=label,
        #color='red',
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Mean distance between venues in cluster:

In [28]:
df_cluster

,Lat,Lng
0,50.061703,19.935660
1,50.063866,19.937492
2,50.061442,19.938733
3,50.062130,19.938760
4,50.062189,19.939250
5,50.046237,19.949234
6,50.062661,19.934744
7,50.063970,19.937753
8,50.083938,19.996193
9,50.061266,19.942233


In [29]:
distance_cracow = pdist(df_cluster, 'cityblock')
print('The mean Manhattan distance between points equals ' + str(distance_cracow.mean()))

The mean Manhattan distance between points equals 0.026348851534691402


<h3>Poznan </h3>

In [30]:
df_cluster = df_poznan.drop(['Name','Address', 'VenueID'], 1)
kclusters = 4
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_cluster)
kmeans.labels_[0:13]

array([3, 0, 2, 1, 0])

In [31]:
df_poznan.insert(0, 'Cluster Labels', kmeans.labels_)
df_poznan.head(2)

,Cluster Labels,Name,Address,VenueID,Lat,Lng
0,3,Blue Note,Kościuszki 79,4c93f79872dd224b349c9591,52.408352,16.919411
1,0,U Bazyla,Norwida 18a,4e18aa3c2271b0e971fea995,52.415897,16.913159


In [32]:
# create map
latitude=df_poznan['Lat'].mean()
longitude=df_poznan['Lng'].mean()
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=14)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(df_poznan['Lat'], df_poznan['Lng'],  df_poznan['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [33]:
df_poznan['Cluster Labels'].value_counts()

0    2
3    1
2    1
1    1
Name: Cluster Labels, dtype: int64

We will consider Cluster 0

In [34]:
df_poznan_fin = df_poznan.loc[df_poznan['Cluster Labels'] == 0]

In [35]:
df_poznan_fin

,Cluster Labels,Name,Address,VenueID,Lat,Lng
1,0,U Bazyla,Norwida 18a,4e18aa3c2271b0e971fea995,52.415897,16.913159
4,0,U Bazyla,NaN,57ffbc5938faf6f1b8bd00c9,52.416519,16.911341


In [36]:
# create map
latitude=df_poznan_fin['Lat'].mean()
longitude=df_poznan_fin['Lng'].mean()
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=16)

# set color scheme for the clusters
#x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon in zip(df_poznan_fin['Lat'], df_poznan_fin['Lng']):
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        #popup=label,
        #color='red',
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [37]:
distance_poznan = pdist(df_cluster, 'cityblock')

In [38]:
print('The mean Manhattan distance between points equals ' + str(distance_poznan.mean()))

The mean Manhattan distance between points equals 0.025583005990241502


<h2>Warsaw</h2>

In [39]:
df_cluster = df_warsaw.drop(['Name','Address', 'VenueID'], 1)
kclusters = 4
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_cluster)
kmeans.labels_[0:13]

array([3, 3, 3, 1, 0, 1, 3, 0, 2, 2, 0, 3, 0])

In [40]:
df_warsaw.insert(0, 'Cluster Labels', kmeans.labels_)
df_warsaw.head(2)

,Cluster Labels,Name,Address,VenueID,Lat,Lng
0,3,miejsce Chwila,Żurawia 47,4ca4d4a75720b1f7db6d3def,52.227504,21.011277
1,3,Hard Rock Cafe Warsaw,"Ul. Zlota 59, Złote Tarasy, Mazovia Province",4b8d7690f964a5202ffd32e3,52.230041,21.003797


In [41]:
# create map
latitude=df_warsaw['Lat'].mean()
longitude=df_warsaw['Lng'].mean()
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(df_warsaw['Lat'], df_warsaw['Lng'],  df_warsaw['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [42]:
df_warsaw['Cluster Labels'].value_counts()

0    7
3    5
2    2
1    2
Name: Cluster Labels, dtype: int64

Cluster 3 will be considered, since while cluster 0 has more venues, it's also more spread out.

In [43]:
df_warsaw_fin = df_warsaw.loc[df_warsaw['Cluster Labels'] == 3]
df_warsaw_fin

,Cluster Labels,Name,Address,VenueID,Lat,Lng
0,3,miejsce Chwila,Żurawia 47,4ca4d4a75720b1f7db6d3def,52.227504,21.011277
1,3,Hard Rock Cafe Warsaw,"Ul. Zlota 59, Złote Tarasy, Mazovia Province",4b8d7690f964a5202ffd32e3,52.230041,21.003797
2,3,12on14 Jazz Club,Noakowskiego 16,552e56de498ea54019222068,52.221727,21.010365
6,3,Proxima,Żwirki i Wigury 99,4cb74dd0f6616dcb07fce508,52.212588,20.987147
11,3,Jazzarium Cafe,Wilcza 50/52,4eac58829adfbd5e5debe7dd,52.224138,21.012222


In [44]:
# create map
latitude=df_warsaw_fin['Lat'].mean()
longitude=df_warsaw_fin['Lng'].mean()
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
#x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon in zip(df_warsaw_fin['Lat'], df_warsaw_fin['Lng']):
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        #popup=label,
        #color='red',
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [45]:
distance_warsaw = pdist(df_cluster, 'cityblock')
print('The mean Manhattan distance between points equals ' + str(distance_warsaw.mean()))

The mean Manhattan distance between points equals 0.05227033952432378


<h2>Wroclaw</h2>

In [46]:
df_cluster = df_wroclaw.drop(['Name','Address', 'VenueID'], 1)
kclusters = 2
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_cluster)
kmeans.labels_[0:13]

array([0, 0, 0, 1, 0, 0])

In [47]:
df_wroclaw.insert(0, 'Cluster Labels', kmeans.labels_)
df_wroclaw.head(2)

,Cluster Labels,Name,Address,VenueID,Lat,Lng
0,0,Stary Klasztor,Purkyniego 1,534d7cac498e13239c12165c,51.109994,17.039282
1,0,Hard Rock Cafe Wroclaw,Rynek 25,5ab3ffaf12f0a96fa8094726,51.109121,17.032220


In [48]:
# create map
latitude=df_wroclaw['Lat'].mean()
longitude=df_wroclaw['Lng'].mean()
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(df_wroclaw['Lat'], df_wroclaw['Lng'],  df_wroclaw['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [49]:
df_wroclaw['Cluster Labels'].value_counts()

0    5
1    1
Name: Cluster Labels, dtype: int64

Cluster 0 will be considered

In [50]:
df_wroclaw_fin = df_wroclaw.loc[df_wroclaw['Cluster Labels'] == 0]
df_wroclaw_fin

,Cluster Labels,Name,Address,VenueID,Lat,Lng
0,0,Stary Klasztor,Purkyniego 1,534d7cac498e13239c12165c,51.109994,17.039282
1,0,Hard Rock Cafe Wroclaw,Rynek 25,5ab3ffaf12f0a96fa8094726,51.109121,17.032220
2,0,Vertigo,Oławska 13,54e10b6f498edfbffd4e1f88,51.108448,17.035366
4,0,Collosseum Jazz Caffe,Piotra Skargi 18a,4ecfe76a4690a03b5cc22e38,51.104830,17.038810
5,0,Liverpool Klub Muzyczny,Świdnicka 37,4e2a10bbb3ade3b5be082dcd,51.104076,17.030501


In [51]:
# create map
latitude=df_wroclaw_fin['Lat'].mean()
longitude=df_wroclaw_fin['Lng'].mean()
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

# set color scheme for the clusters
#x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon in zip(df_wroclaw_fin['Lat'], df_wroclaw_fin['Lng']):
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        #popup=label,
        #color='red',
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [52]:
distance_wroclaw = pdist(df_cluster, 'cityblock')
print('The mean Manhattan distance between points equals ' + str(distance_wroclaw.mean()))

The mean Manhattan distance between points equals 0.021009871901528736


<h2>Conclusions</h2>

In [53]:
print('Number of selected venues in Cracow ' + str(df_cracow_fin.shape[0]) + ' with mean distance ' + str(distance_cracow.mean()))
print('Number of selected venues in Poznan ' + str(df_poznan_fin.shape[0]) + ' with mean distance ' + str(distance_poznan.mean()))
print('Number of selected venues in Warsaw ' + str(df_warsaw_fin.shape[0]) + ' with mean distance ' + str(distance_warsaw.mean()))
print('Number of selected venues in Wroclaw ' + str(df_wroclaw_fin.shape[0]) + ' with mean distance ' + str(distance_wroclaw.mean()))

Number of selected venues in Cracow 9 with mean distance 0.026348851534691402
Number of selected venues in Poznan 2 with mean distance 0.025583005990241502
Number of selected venues in Warsaw 5 with mean distance 0.05227033952432378
Number of selected venues in Wroclaw 5 with mean distance 0.021009871901528736


Poznan has the least ammount of venues. Warsaw is similar to Wroclaw, but in Wroclaw the venues arent spread as much. Mean distance is similar in Wroclaw and Cracow, but Cracow has almost twice as many venues as Wroclaw. Thus, Cracow is the recommended city.